In [1]:
import os
import sys

os.environ["SPARK_HOME"] = "/opt/spark"
spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib'))

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [7]:
spark.read.csv("/home/orpington/Desktop/dbgen/tpch-100/*").count()

KeyboardInterrupt: 

In [3]:
s = """
CREATE TABLE [dbo].[CUSTOMER](
	[C_CUSTKEY] [int] NOT NULL,
	[C_NAME] [varchar](25) NOT NULL,
	[C_ADDRESS] [varchar](40) NOT NULL,
	[C_NATIONKEY] [int] NOT NULL,
	[C_PHONE] [char](15) NOT NULL,
	[C_ACCTBAL] [decimal](15, 2) NOT NULL,
	[C_MKTSEGMENT] [char](10) NOT NULL,
	[C_COMMENT] [varchar](117) NOT NULL
);
GO
CREATE TABLE [dbo].[LINEITEM](
	[L_ORDERKEY] [int] NOT NULL,
	[L_PARTKEY] [int] NOT NULL,
	[L_SUPPKEY] [int] NOT NULL,
	[L_LINENUMBER] [int] NOT NULL,
	[L_QUANTITY] [decimal](15, 2) NOT NULL,
	[L_EXTENDEDPRICE] [decimal](15, 2) NOT NULL,
	[L_DISCOUNT] [decimal](15, 2) NOT NULL,
	[L_TAX] [decimal](15, 2) NOT NULL,
	[L_RETURNFLAG] [char](1) NOT NULL,
	[L_LINESTATUS] [char](1) NOT NULL,
	[L_SHIPDATE] [date] NOT NULL,
	[L_COMMITDATE] [date] NOT NULL,
	[L_RECEIPTDATE] [date] NOT NULL,
	[L_SHIPINSTRUCT] [char](25) NOT NULL,
	[L_SHIPMODE] [char](10) NOT NULL,
	[L_COMMENT] [varchar](44) NOT NULL
);
GO
CREATE TABLE [dbo].[NATION](
	[N_NATIONKEY] [int] NOT NULL,
	[N_NAME] [char](25) NOT NULL,
	[N_REGIONKEY] [int] NOT NULL,
	[N_COMMENT] [varchar](152) NULL
);
GO
CREATE TABLE [dbo].[ORDERS](
	[O_ORDERKEY] [int] NOT NULL,
	[O_CUSTKEY] [int] NOT NULL,
	[O_ORDERSTATUS] [char](1) NOT NULL,
	[O_TOTALPRICE] [decimal](15, 2) NOT NULL,
	[O_ORDERDATE] [date] NOT NULL,
	[O_ORDERPRIORITY] [char](15) NOT NULL,
	[O_CLERK] [char](15) NOT NULL,
	[O_SHIPPRIORITY] [int] NOT NULL,
	[O_COMMENT] [varchar](79) NOT NULL
);
GO
CREATE TABLE [dbo].[PART](
	[P_PARTKEY] [int] NOT NULL,
	[P_NAME] [varchar](55) NOT NULL,
	[P_MFGR] [char](25) NOT NULL,
	[P_BRAND] [char](10) NOT NULL,
	[P_TYPE] [varchar](25) NOT NULL,
	[P_SIZE] [int] NOT NULL,
	[P_CONTAINER] [char](10) NOT NULL,
	[P_RETAILPRICE] [decimal](15, 2) NOT NULL,
	[P_COMMENT] [varchar](23) NOT NULL
);
GO
CREATE TABLE [dbo].[PARTSUPP](
	[PS_PARTKEY] [int] NOT NULL,
	[PS_SUPPKEY] [int] NOT NULL,
	[PS_AVAILQTY] [int] NOT NULL,
	[PS_SUPPLYCOST] [decimal](15, 2) NOT NULL,
	[PS_COMMENT] [varchar](199) NOT NULL
);
GO
CREATE TABLE [dbo].[REGION](
	[R_REGIONKEY] [int] NOT NULL,
	[R_NAME] [char](25) NOT NULL,
	[R_COMMENT] [varchar](152) NULL
);
GO
CREATE TABLE [dbo].[SUPPLIER](
	[S_SUPPKEY] [int] NOT NULL,
	[S_NAME] [char](25) NOT NULL,
	[S_ADDRESS] [varchar](40) NOT NULL,
	[S_NATIONKEY] [int] NOT NULL,
	[S_PHONE] [char](15) NOT NULL,
	[S_ACCTBAL] [decimal](15, 2) NOT NULL,
	[S_COMMENT] [varchar](101) NOT NULL
);
GO
"""

In [53]:
splt = s.split("GO")[0]

In [55]:
print(splt)


CREATE TABLE [dbo].[CUSTOMER](
	[C_CUSTKEY] [int] NOT NULL,
	[C_NAME] [varchar](25) NOT NULL,
	[C_ADDRESS] [varchar](40) NOT NULL,
	[C_NATIONKEY] [int] NOT NULL,
	[C_PHONE] [char](15) NOT NULL,
	[C_ACCTBAL] [decimal](15, 2) NOT NULL,
	[C_MKTSEGMENT] [char](10) NOT NULL,
	[C_COMMENT] [varchar](117) NOT NULL
);



In [56]:
import re
re.search(r"[\[][^\]]*[\]][(]", splt).group(0)

'[CUSTOMER]('

In [57]:
re.search(r"(?<=[\]][(]).+", splt).group(0)

'25) NOT NULL,'

In [63]:
re.search(r"[^\]\[]*\]\(", splt).group(0)

'CUSTOMER]('

In [71]:
name_find = r"[^\]\[]+(?=\]\()"
re.search(name_find, splt).group(0)

'CUSTOMER'

In [72]:
splt.split(",")

['\nCREATE TABLE [dbo].[CUSTOMER](\n\t[C_CUSTKEY] [int] NOT NULL',
 '\n\t[C_NAME] [varchar](25) NOT NULL',
 '\n\t[C_ADDRESS] [varchar](40) NOT NULL',
 '\n\t[C_NATIONKEY] [int] NOT NULL',
 '\n\t[C_PHONE] [char](15) NOT NULL',
 '\n\t[C_ACCTBAL] [decimal](15',
 ' 2) NOT NULL',
 '\n\t[C_MKTSEGMENT] [char](10) NOT NULL',
 '\n\t[C_COMMENT] [varchar](117) NOT NULL\n);\n']

In [77]:
for line in re.split(r"\n\t\[", splt)[1:]:
    print(line.split("]")[0])

C_CUSTKEY
C_NAME
C_ADDRESS
C_NATIONKEY
C_PHONE
C_ACCTBAL
C_MKTSEGMENT
C_COMMENT


In [5]:
#https://pymotw.com/2/collections/counter.html
import re
from collections import Counter
counter = Counter()

with open("/home/orpington/Desktop/dbgen/schema.csv", "w") as f:
    for table_schema in s.split("GO"):
        table = re.search(r"[^\]\[]+(?=\]\()", table_schema)
        if table:
            table_name = table.group(0)
            for line in re.split(r"\n\t\[", table_schema)[1:]:
                counter.update([table_name])
                f.write("{},{},{}\n".format(
                                            counter[table_name], 
                                            table_name, 
                                            line.split("]")[0].split("_")[1]))

In [99]:
f = open("/home/orpington/Desktop/dbgen/schema.csv", "r")
print(repr(f.readlines()))
f.close()

['1,CUSTOMER,CUSTKEY\n', '2,CUSTOMER,NAME\n', '3,CUSTOMER,ADDRESS\n', '4,CUSTOMER,NATIONKEY\n', '5,CUSTOMER,PHONE\n', '6,CUSTOMER,ACCTBAL\n', '7,CUSTOMER,MKTSEGMENT\n', '8,CUSTOMER,COMMENT\n', '1,LINEITEM,ORDERKEY\n', '2,LINEITEM,PARTKEY\n', '3,LINEITEM,SUPPKEY\n', '4,LINEITEM,LINENUMBER\n', '5,LINEITEM,QUANTITY\n', '6,LINEITEM,EXTENDEDPRICE\n', '7,LINEITEM,DISCOUNT\n', '8,LINEITEM,TAX\n', '9,LINEITEM,RETURNFLAG\n', '10,LINEITEM,LINESTATUS\n', '11,LINEITEM,SHIPDATE\n', '12,LINEITEM,COMMITDATE\n', '13,LINEITEM,RECEIPTDATE\n', '14,LINEITEM,SHIPINSTRUCT\n', '15,LINEITEM,SHIPMODE\n', '16,LINEITEM,COMMENT\n', '1,NATION,NATIONKEY\n', '2,NATION,NAME\n', '3,NATION,REGIONKEY\n', '4,NATION,COMMENT\n', '1,ORDERS,ORDERKEY\n', '2,ORDERS,CUSTKEY\n', '3,ORDERS,ORDERSTATUS\n', '4,ORDERS,TOTALPRICE\n', '5,ORDERS,ORDERDATE\n', '6,ORDERS,ORDERPRIORITY\n', '7,ORDERS,CLERK\n', '8,ORDERS,SHIPPRIORITY\n', '9,ORDERS,COMMENT\n', '1,PART,PARTKEY\n', '2,PART,NAME\n', '3,PART,MFGR\n', '4,PART,BRAND\n', '5,PART,TY

In [29]:
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType

csv_schema = spark.read\
                   .csv("/home/orpington/Desktop/dbgen/schema.csv")\
                   .where(col("_c1") == "LINEITEM")\
                   .rdd\
                   .map(lambda row: StructField(row[2], StringType(), False))\
                   .collect()
StructType(csv_schema)

StructType(List(StructField(ORDERKEY,StringType,false),StructField(PARTKEY,StringType,false),StructField(SUPPKEY,StringType,false),StructField(LINENUMBER,StringType,false),StructField(QUANTITY,StringType,false),StructField(EXTENDEDPRICE,StringType,false),StructField(DISCOUNT,StringType,false),StructField(TAX,StringType,false),StructField(RETURNFLAG,StringType,false),StructField(LINESTATUS,StringType,false),StructField(SHIPDATE,StringType,false),StructField(COMMITDATE,StringType,false),StructField(RECEIPTDATE,StringType,false),StructField(SHIPINSTRUCT,StringType,false),StructField(SHIPMODE,StringType,false),StructField(COMMENT,StringType,false)))

In [28]:
data = spark.read\
            .format("csv")\
            .option("schema", StructType(csv_schema))\
            .option("sep", "|")\
            .load("/home/orpington/Desktop/dbgen/tpch-1/lineitem.tbl")

data.show()

+---+------+----+---+---+--------+----+----+---+---+----------+----------+----------+-----------------+-------+--------------------+----+
|_c0|   _c1| _c2|_c3|_c4|     _c5| _c6| _c7|_c8|_c9|      _c10|      _c11|      _c12|             _c13|   _c14|                _c15|_c16|
+---+------+----+---+---+--------+----+----+---+---+----------+----------+----------+-----------------+-------+--------------------+----+
|  1|155190|7706|  1| 17|21168.23|0.04|0.02|  N|  O|1996-03-13|1996-02-12|1996-03-22|DELIVER IN PERSON|  TRUCK|egular courts abo...|null|
|  1| 67310|7311|  2| 36|45983.16|0.09|0.06|  N|  O|1996-04-12|1996-02-28|1996-04-20| TAKE BACK RETURN|   MAIL|ly final dependen...|null|
|  1| 63700|3701|  3|  8|13309.60|0.10|0.02|  N|  O|1996-01-29|1996-03-05|1996-01-31| TAKE BACK RETURN|REG AIR|riously. regular,...|null|
|  1|  2132|4633|  4| 28|28955.64|0.09|0.06|  N|  O|1996-04-21|1996-03-30|1996-05-16|             NONE|    AIR|lites. fluffily e...|null|
|  1| 24027|1534|  5| 24|22824.48|

In [30]:
data = spark.read\
            .csv("/home/orpington/Desktop/dbgen/tpch-1/lineitem.tbl", 
                 schema=StructType(csv_schema),
                 sep="|"
                )

data.show()

+--------+-------+-------+----------+--------+-------------+--------+----+----------+----------+----------+----------+-----------+-----------------+--------+--------------------+
|ORDERKEY|PARTKEY|SUPPKEY|LINENUMBER|QUANTITY|EXTENDEDPRICE|DISCOUNT| TAX|RETURNFLAG|LINESTATUS|  SHIPDATE|COMMITDATE|RECEIPTDATE|     SHIPINSTRUCT|SHIPMODE|             COMMENT|
+--------+-------+-------+----------+--------+-------------+--------+----+----------+----------+----------+----------+-----------+-----------------+--------+--------------------+
|       1| 155190|   7706|         1|      17|     21168.23|    0.04|0.02|         N|         O|1996-03-13|1996-02-12| 1996-03-22|DELIVER IN PERSON|   TRUCK|egular courts abo...|
|       1|  67310|   7311|         2|      36|     45983.16|    0.09|0.06|         N|         O|1996-04-12|1996-02-28| 1996-04-20| TAKE BACK RETURN|    MAIL|ly final dependen...|
|       1|  63700|   3701|         3|       8|     13309.60|    0.10|0.02|         N|         O|1996-01-2

In [31]:
data.count()

6001215

In [47]:
from pyspark.sql.functions import input_file_name, regexp_extract
data = spark.read\
            .csv("/home/orpington/Desktop/dbgen/tpch-1/*.tbl")\
            .groupby(input_file_name())\
            .count()
data.show(20, False)

+--------------------------------------------------------+-------+
|input_file_name()                                       |count  |
+--------------------------------------------------------+-------+
|file:///home/orpington/Desktop/dbgen/tpch-1/lineitem.tbl|6001215|
|file:///home/orpington/Desktop/dbgen/tpch-1/customer.tbl|150000 |
|file:///home/orpington/Desktop/dbgen/tpch-1/supplier.tbl|10000  |
|file:///home/orpington/Desktop/dbgen/tpch-1/partsupp.tbl|800000 |
|file:///home/orpington/Desktop/dbgen/tpch-1/nation.tbl  |25     |
|file:///home/orpington/Desktop/dbgen/tpch-1/region.tbl  |5      |
|file:///home/orpington/Desktop/dbgen/tpch-1/part.tbl    |200000 |
|file:///home/orpington/Desktop/dbgen/tpch-1/orders.tbl  |1500000|
+--------------------------------------------------------+-------+



In [60]:
from pyspark.sql.functions import input_file_name, split, reverse, size
data = spark.read\
            .csv("/home/orpington/Desktop/dbgen/tpch-1/*.tbl")\
            .withColumn("split", split(input_file_name(), r"/"))\
            .groupby(col("split")[size(col("split")) - 1].alias("table"))\
            .count()

data.show(20, False)

+------------+-------+
|table       |count  |
+------------+-------+
|lineitem.tbl|6001215|
|nation.tbl  |25     |
|supplier.tbl|10000  |
|customer.tbl|150000 |
|orders.tbl  |1500000|
|region.tbl  |5      |
|part.tbl    |200000 |
|partsupp.tbl|800000 |
+------------+-------+



In [65]:
csv = spark.read.option("sep", "|")\
                .format("csv")\
                .option("mode", "PERMISSIVE")\
                .load("/home/orpington/Desktop/dbgen/tpch-1/region_bad.tbl")
csv.show()

+--------------------+-----------+--------------------+----+
|                 _c0|        _c1|                 _c2| _c3|
+--------------------+-----------+--------------------+----+
|                   0|     AFRICA|lar deposits. bli...|null|
|                   1|    AMERICA|hs use ironic, ev...|null|
|                   2|       ASIA|ges. thinly even ...|null|
|                   3|     EUROPE|ly final courts c...|null|
|                   4|MIDDLE EAST|uickly special ac...|null|
|fdfdf bad rcrdssdssd|       null|                null|null|
+--------------------+-----------+--------------------+----+



In [71]:
csv = spark.read.option("sep", "|")\
            .format("csv")\
            .option("mode", "FAILFAST")\
            .load("/home/orpington/Desktop/dbgen/tpch-1/region_bad.tbl")
try:
    csv.show()
except:
    print("error!")

error!


In [77]:
csv = spark.read.option("sep", "|")\
                .format("csv")\
                .option("mode", "DROPMALFORMED")\
                .load("/home/orpington/Desktop/dbgen/tpch-1/region_bad.tbl")
csv.show()

+---+-----------+--------------------+----+
|_c0|        _c1|                 _c2| _c3|
+---+-----------+--------------------+----+
|  0|     AFRICA|lar deposits. bli...|null|
|  1|    AMERICA|hs use ironic, ev...|null|
|  2|       ASIA|ges. thinly even ...|null|
|  3|     EUROPE|ly final courts c...|null|
|  4|MIDDLE EAST|uickly special ac...|null|
+---+-----------+--------------------+----+



In [89]:
for _ in range(10):
    csv.write.csv("/tmp/herp2.txt", mode="append")

spark.read.csv("/tmp/herp2.txt").show()

+--------------------+--------------------+
|                 _c0|                 _c1|
+--------------------+--------------------+
|          PAR1 >@|                   |
| 40   ...|                   |
| MIDDLE E...|                null|
|ME SIAPE...|                   |
| luickly s...|                null|
|        #account	%|                null|
||rding to    hs ...| evenD s#(ges. ...|
|           �court|jole furious\ex...|
|             6|                null|
|L&5 ...|                null|
|tv&<40  ...|                null|
|��&~<MIDDLE...|                null|
|��&�<luickl...|                null|
|           6:&�	<6|                null|
|                   0|              AFRICA|
|                   1|             AMERICA|
|                   2|                ASIA|
|                   3|              EUROPE|
|                   4|         MIDDLE EAST|
|                   0|          

In [90]:
for _ in range(10):
    csv.write.csv("/tmp/herp2.txt", mode="overwrite")

spark.read.csv("/tmp/herp2.txt").show()

+---+-----------+--------------------+----+
|_c0|        _c1|                 _c2| _c3|
+---+-----------+--------------------+----+
|  0|     AFRICA|lar deposits. bli...|null|
|  1|    AMERICA|hs use ironic, ev...|null|
|  2|       ASIA|ges. thinly even ...|null|
|  3|     EUROPE|ly final courts c...|null|
|  4|MIDDLE EAST|uickly special ac...|null|
+---+-----------+--------------------+----+



In [91]:
for _ in range(10):
    csv.write.csv("/tmp/herp2.txt", mode="errorIfExists")

spark.read.csv("/tmp/herp2.txt").show()

IllegalArgumentException: u"Unknown save mode: errorIfExists. Accepted save modes are 'overwrite', 'append', 'ignore', 'error'."

In [92]:
for _ in range(10):
    csv.write.csv("/tmp/herp2.txt", mode="ignore")

spark.read.csv("/tmp/herp2.txt").show()

+---+-----------+--------------------+----+
|_c0|        _c1|                 _c2| _c3|
+---+-----------+--------------------+----+
|  0|     AFRICA|lar deposits. bli...|null|
|  1|    AMERICA|hs use ironic, ev...|null|
|  2|       ASIA|ges. thinly even ...|null|
|  3|     EUROPE|ly final courts c...|null|
|  4|MIDDLE EAST|uickly special ac...|null|
+---+-----------+--------------------+----+



In [93]:
spark.read.csv("/tmp/herp2.txt").printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)



In [103]:
spark.read\
     .csv("/home/orpington/Desktop/herp.txt", inferSchema=True)\
     .printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)



In [104]:
spark.read.csv("/home/orpington/Desktop/herp.txt", escape=" ").show() #??

+---+-------------+--------------------+----+
|_c0|          _c1|                 _c2| _c3|
+---+-------------+--------------------+----+
|  0|       AFRICA|lar deposits. bli...|null|
|  1|      AMERICA|hsuseironic,evenr...|null|
|  2|         ASIA|ges. thinly even ...|null|
|  3|   EUROPE    |ly final courts c...|null|
|  4|  MIDDLE EAST|uickly special ac...|null|
+---+-------------+--------------------+----+



In [106]:
spark.read.csv("/home/orpington/Desktop/herp.txt", ignoreLeadingWhiteSpace=True, ignoreTrailingWhiteSpace=True).show() 

+---+-----------+--------------------+----+
|_c0|        _c1|                 _c2| _c3|
+---+-----------+--------------------+----+
|  0|     AFRICA|lar deposits. bli...|null|
|  1|    AMERICA|hs use ironic, ev...|null|
|  2|       ASIA|ges. thinly even ...|null|
|  3|     EUROPE|ly final courts c...|null|
|  4|MIDDLE EAST|uickly special ac...|null|
+---+-----------+--------------------+----+



In [113]:
spark.read.csv("/home/orpington/Desktop/herp.txt",  nullValue=1, positiveInf=2).show() 

+----+-------------+--------------------+----+
| _c0|          _c1|                 _c2| _c3|
+----+-------------+--------------------+----+
|   0|       AFRICA|lar deposits. bli...|null|
|null|      AMERICA|hs use ironic, ev...|null|
|   2|         ASIA|ges. thinly even ...|null|
|   3|   EUROPE    |ly final courts c...|null|
|   4|  MIDDLE EAST|uickly special ac...|null|
+----+-------------+--------------------+----+



In [114]:
csv.write.json("/home/orpington/Desktop/herp3.txt")

In [115]:
csv.write.json("/home/orpington/Desktop/herp3.txt", wholeFile=True)

TypeError: json() got an unexpected keyword argument 'wholeFile'